In [3]:
import os 
import sys
from ase import Atoms

sys.path.append('../pgm/CCS/')
 
from ase import io

from ase.optimize import FIRE
from ase.io import read, write
import numpy as np
import pinn
from ase.calculators.dftb import Dftb
from ase.calculators.mixing import LinearCombinationCalculator
import json
from ccs.ase_tools.ccs_ase_calculator import CCS  
import ase.db as db

base_dir=os.getcwd()
os.chdir(base_dir)


if not os.path.isdir(base_dir+"/RUN_ModelAnalysis"):
    os.mkdir(base_dir+"/RUN_ModelAnalysis/")

def setup_model(spin,charge,**kwargs):

    # set up model:
    # 1: DFTB 
    os.environ["DFTB_PREFIX"] = base_dir+"/SKF/"
    DFTB_calc=Dftb(Hamiltonian_SCC='Yes',Hamiltonian_ShellResolvedSCC = 'Yes',Hamiltonian_SCCTolerance = '1.0E-006',Hamiltonian_Mixer= 'Anderson{',Hamiltonian_Mixer_MixingParameter='0.03',Hamiltonian_MaxSCCIterations = '1500',Hamiltonian_Filling = 'MethfesselPaxton{',Hamiltonian_Filling_Temperature = '0.001583407672623195',Hamiltonian_KPointsAndWeights = 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}') 
    if spin>0:                  
        DFTB_calc.set(Hamiltonian_SpinPolarisation = 'Colinear{') 
        DFTB_calc.set(Hamiltonian_SpinPolarisation_UnpairedElectrons=str(spin))
        DFTB_calc.set(Hamiltonian_SpinConstants_ ='')
        for i in elements: 
            print(i)
            spinparam=get_spindata(i)
            string=f'Hamiltonian_SpinConstants_{i}'  
            print(string)
            DFTB_calc.set(**{string:spinparam})  
    if charge>0: 
        DFTB_calc.set(Hamiltonian_Charge =str(charge))
    print(DFTB_calc.todict())     

                     
    #2: CCS 
    with open(base_dir+'/CCS/CCS_params.json', 'r') as f:
        CCS_params = json.load(f)

    CCS_calc=CCS(CCS_params=CCS_params) 
    #3: PiNN 
    PiNN_calc = pinn.get_calc(base_dir+'/PiNN/') 
    

    # and the calculater becomes 
    calcs =[DFTB_calc,CCS_calc, PiNN_calc]
    weights=[1,1,1]
    calc=LinearCombinationCalculator(calcs, weights) 
    return calc


def get_data(element): 
    if element=='Li':
        maxang="'p'"
    if element=='C':
        maxang="'p'"
    if element=='O':
        maxang="'p'"
    if element=='H':
        maxang="'s'"
    if element=='P':
        maxang="'p'"    
    return maxang                 

def get_spindata(element): 
    if element=='Li':
        spinparam='{-0.019  -0.016  -0.016 -0.027}'
    if element=='C':
        spinparam='{-0.031 -0.025 -0.025 -0.023}'
    if element=='O':
        spinparam='{-0.035 -0.030 -0.030 -0.028}'
    if element=='H':
        spinparam='{-0.072}'
    if element=='Si':
        spinparam='{-0.035 -0.030 -0.030 -0.0280}'   
    if element=='P':
        spinparam='{-0.035 -0.030 -0.030 -0.0280}'  
    return spinparam        


In [ ]:

dbname_DFT='DFT_DB.db'
dbname_DFTB='DFTB_DB.db'
db_dftb=db.connect(dbname_DFTB)
db_dft=db.connect(dbname_DFT)

os.chdir(base_dir+"/RUN_ModelAnalysis/")

#idlist=create_id_list()  
#loc=np.random.randint(1,21,21)
#print(loc)
spin=0
charge=0

epred=[]; etrue=[]
for row in db_dftb.select():
    structure=row.toatoms()
    natoms=len(structure.get_atomic_numbers())
    id=str(row.key)
    structure.calc=setup_model(spin, charge)
    try: 
        epred.append(structure.get_potential_energy()/natoms)
        etrue.append(db_dft.get('key='+id).energy/natoms)
    except:
        print('Error')
        
os.chdir(base_dir)

{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}


/home/broqvist/miniconda3/envs/parautomatik/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}


/home/broqvist/miniconda3/envs/parautomatik/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.rank instead
  warnings.warn(warning)


{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}


...

Error
{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}


OpenBLAS blas_thread_init: pthread_create failed for thread 35 of 72: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1031161 current, 1031161 max


Error
{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}


OpenBLAS blas_thread_init: pthread_create failed for thread 35 of 72: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1031161 current, 1031161 max


Error
{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}


In [10]:
print(epred,etrue)

[-56.75669323099061, -56.54752196334687, -56.43233726549806, -55.76038992119919, -56.750803915364315, -55.21102575886289, -54.837298655806705, -55.888424180041795] [-56.7489275, -55.99074181, -55.89346293, -55.91594481, -56.21340322, -55.91404, -55.54594199, -56.22496852]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (10,10))
sns.scatterplot(epred,etrue, s=100, alpha=0.4).set_title('Correlation predicted vs DFT energy', fontsize=15)
plt.plot([np.min(epred),np.max(epred)], [np.min(etrue),np.max(etrue)], 'k-', lw=0.5)
plt.legend()
plt.xlabel('E_DFT')
plt.ylabel('E_model')
plt.xlim(-5.7,-5.4)
plt.ylim(-5.7,-5.4)
plt.show()